# Explore Calibrated Data

In [ ]:
import ctapipe
from ctapipe.utils.datasets import get_dataset_path
from ctapipe.io import EventSource, EventSeeker
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from matplotlib import pyplot as plt
from astropy import units as u
import numpy as np

%matplotlib inline
plt.style.use("ggplot")

In [ ]:
print(ctapipe.__version__)
print(ctapipe.__file__)

Let's first open a raw event file and get an event out of it:

In [ ]:
filename = get_dataset_path("gamma_prod5.simtel.zst")
source = EventSource(filename, max_events=2)

for event in source:
    print(event.index.event_id)

In [ ]:
filename

In [ ]:
source

In [ ]:
event

In [ ]:
event.tel[26]

## Perform basic calibration:

Here we will use a `CameraCalibrator` which is just a simple wrapper that runs the two calibration and trace-integration phases of the pipeline, taking the data from levels:

  **R1** &rightarrow; **DL0** &rightarrow; **DL1**

Note that we have not specified any configuration to the `CameraCalibrator`, so it will be using the default algorithms and thresholds.

In [ ]:
from ctapipe.calib import CameraCalibrator

calib = CameraCalibrator(subarray=source.subarray)
calib(event)

Now the *r1*, *dl0* and *dl1* containers are filled in the event

* **r0**: Contains device specific raw data. This is usually only available in simulations or in expert data. 
* **r1**: contains the "r1-calibrated" waveforms, after gain-selection, pedestal subtraction, and gain-correction
* **dl0**: is the same but with optional data volume reduction (some pixels not filled), by default, this is not performed, so it is the same as r1
* **dl1**: contains the time-integrated *image* that has been calculated using an `ImageExtractor` (`NeighborPeakWindowSum` by default)

In [ ]:
for tel_id, tel_event in event.tel.items():
    print("TEL{:03}: {}".format(tel_id, source.subarray.tel[tel_id]))
    print("  - r0  wave shape  : {}".format(tel_event.r0.waveform.shape))
    print("  - r1  wave shape  : {}".format(tel_event.r1.waveform.shape))
    print("  - dl1 image shape : {}".format(tel_event.dl1.image.shape))

## Some image processing:

Let's look at the image

In [ ]:
from ctapipe.visualization import CameraDisplay

tel_id, tel_event = next(iter(event.tel.items()))
sub = source.subarray
geometry = sub.tel[tel_id].camera.geometry
image = tel_event.dl1.image

In [ ]:
disp = CameraDisplay(geometry, image=image)

In [ ]:
from ctapipe.image import tailcuts_clean, hillas_parameters

In [ ]:
mask = tailcuts_clean(
    geometry,
    image,
    picture_thresh=10,
    boundary_thresh=5,
    min_number_picture_neighbors=2,
)
cleaned = image.copy()
cleaned[~mask] = 0
disp = CameraDisplay(geometry, image=cleaned)

In [ ]:
params = hillas_parameters(geometry, cleaned)
print(params)
params

In [ ]:
params = hillas_parameters(geometry, cleaned)

plt.figure(figsize=(5, 5))
disp = CameraDisplay(geometry, image=image)
disp.add_colorbar()
disp.overlay_moments(params, color="xkcd:light blue", lw=3)
disp.highlight_pixels(mask, color="white", alpha=0.3, linewidth=2)

plt.xlim(params.x.to_value(u.m) - 0.5, params.x.to_value(u.m) + 0.5)
plt.ylim(params.y.to_value(u.m) - 0.5, params.y.to_value(u.m) + 0.5)

## ImageProcessor

The above steps can be configured and run easily using the `ImageProcessor` class:

In [ ]:
from ctapipe.image import ImageProcessor

image_processor = ImageProcessor(subarray=source.subarray, use_telescope_frame=False)

image_processor(event)

## More complex image processing:

Let's now explore how stereo reconstruction works. 

### first, look at a summed image from multiple telescopes

For this, we want to use a `CameraDisplay` again, but since we can't sum and display images with different cameras, we'll just sub-select images from a particular camera type

These are the telescopes that are in this event:

In [ ]:
# use a set here, so we can intersect it later
tels_in_event = set(event.tel.keys())
tels_in_event

In [ ]:
mst_tel_ids = set(sub.get_tel_ids_for_type("MST_MST_NectarCam"))
mst_tel_ids

In [ ]:
msts_in_event = list(tels_in_event.intersection(mst_tel_ids))

tel = sub.tel[msts_in_event[0]]
print(f"{tel} in event: {msts_in_event}")

Now let's sum and display those images

In [ ]:
image_sum = np.zeros(tel.camera.geometry.n_pixels)


fig, ax = plt.subplots(figsize=(8, 8))

disp = CameraDisplay(tel.camera.geometry, ax=ax)


for tel_id in msts_in_event:
    dl1 = event.tel[tel_id].dl1
    image_sum += dl1.image

    disp.overlay_moments(
        dl1.parameters.hillas, with_label=False, keep_old=True, lw=3, n_sigma=2
    )

disp.image = image_sum
plt.title("Sum of {}x {}".format(len(msts_in_event), tel))

let's also show which telescopes those were. Note that currently ArrayDisplay's value field is a vector by `tel_index`, not `tel_id`, so we have to convert to a tel_index. (this may change in a future version to be more user-friendly)


In [ ]:
from ctapipe.visualization import ArrayDisplay

In [ ]:
nectarcam_subarray = sub.select_subarray(mst_tel_ids, name="NectarCam")

hit_pattern = np.zeros(shape=nectarcam_subarray.n_tels)
hit_pattern[nectarcam_subarray.tel_ids_to_indices(msts_in_event)] = 1

plt.set_cmap(plt.cm.Accent)
plt.figure(figsize=(8, 8))

ad = ArrayDisplay(nectarcam_subarray)
ad.values = hit_pattern
ad.add_labels()